# Independent Q-learning

> In this notebook, we implement the **[Independent Q learning]()** Multi Agent Reinforcement Leaning (MARL) algorithm in our environment. 


> Tutorial based on [IQL TorchRL Tutorial](https://github.com/pytorch/rl/blob/main/sota-implementations/multiagent/iql.py).

#### High-level overview of IQL algorithm

In IQL a centralized state-action value function is used, Q<sub>tot</sub>, and each agent α learns an individual action-value function Q<sub>α</sub>, independently.

### Simulation overview

> We simulate our environment with an initial population of **200 human agents**. These agents navigate the environment and eventually converge on the fastest path. After this convergence, we will transition **50 of these human agents** into **machine agents**, specifically autonomous vehicles (AVs), which will then employ the Independent Q learning reinforcement learning algorithm to further refine their learning.

#### Imported libraries

In [21]:
%run startup.py

In [22]:
import os
import torch

from tensordict.nn import TensorDictModule, TensorDictSequential
from torchrl.envs.libs.pettingzoo import PettingZooWrapper
from torchrl.envs.transforms import TransformedEnv, RewardSum
from torchrl.envs.utils import check_env_specs
from torch import nn
from torchrl._utils import logger as torchrl_logger
from torchrl.collectors import SyncDataCollector
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.modules import EGreedyModule, QValueModule, SafeSequential
from torchrl.modules.models.multiagent import MultiAgentMLP
from torchrl.objectives import SoftUpdate, ValueEstimators, DQNLoss

from RouteRL import TrafficEnvironment

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [23]:
device = (
    torch.device(0)
    if torch.cuda.is_available()
    else torch.device("cpu")
)

print("device is: ", device)

device is:  cpu


#### Hyperparameters setting

In [24]:
# Sampling
frames_per_batch = 100  # Number of team frames collected per training iteration
n_iters = 50  # Number of sampling and training iterations - the episodes the plotter plots
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 1  # Number of optimization steps per training iteration 
minibatch_size = 16  # Size of the mini-batches in each optimization step
lr = 3e-3 # Learning rate
max_grad_norm = 5.0  # Maximum norm for the gradients
memory_size = 5_000  # Size of the replay buffer
tau =  0.05
gamma = 0.99  # discount factor
exploration_fraction = 1/3 # Fraction of frames over which the exploration rate is annealed

eps = 1 - tau
eps_init = 0.99
eps_end = 0

mlp_depth = 2
mlp_cells = 32

# Human learning phase
human_learning_episodes = 100

# Environment
env_params = {
    "agent_parameters" : {
        "num_agents" : 200,
        "new_machines_after_mutation": 50,
        "human_parameters" : {
            "model" : "w_avg"
        },
    },
    "simulator_parameters" : {
        "network_name" : "csomor"
    },  
    "plotter_parameters" : {
        "phases" : [0, human_learning_episodes],
        "smooth_by" : 50,
    }
}

#### Environment initialization

> In this example, the environment initially contains only human agents.

In [25]:
env = TrafficEnvironment(seed=42, **env_params)
print(env)

[CONFIRMED] Environment variable exists: SUMO_HOME
[SUCCESS] Added module directory: C:\Program Files (x86)\Eclipse\Sumo\tools


KeyboardInterrupt: 

> Reset the environment and the connection with SUMO

In [ ]:
env.start()
env.reset()

({}, {})

#### Human learning

In [ ]:
for episode in range(human_learning_episodes):
    env.step()

self.day is:  0 


self.day is:  1 


self.day is:  2 


self.day is:  3 


self.day is:  4 


self.day is:  5 


self.day is:  6 


self.day is:  7 


self.day is:  8 


self.day is:  9 


self.day is:  10 


self.day is:  11 


self.day is:  12 


self.day is:  13 


self.day is:  14 


self.day is:  15 


self.day is:  16 


self.day is:  17 


self.day is:  18 


self.day is:  19 


self.day is:  20 


self.day is:  21 


self.day is:  22 


self.day is:  23 


self.day is:  24 


self.day is:  25 


self.day is:  26 


self.day is:  27 


self.day is:  28 


self.day is:  29 


self.day is:  30 


self.day is:  31 


self.day is:  32 


self.day is:  33 


self.day is:  34 


self.day is:  35 


self.day is:  36 


self.day is:  37 


self.day is:  38 


self.day is:  39 


self.day is:  40 


self.day is:  41 


self.day is:  42 


self.day is:  43 


self.day is:  44 


self.day is:  45 


self.day is:  46 


self.day is:  47 


self.day is:  48 


self.day is:  49 


self.day i

#### Mutation

> **Mutation**: a portion of human agents are converted into machine agents (autonomous vehicles).

In [ ]:
env.mutation()
print(env)

TrafficEnvironment with 200 agents.            
50 machines and 150 humans.            
Machines: [Machine 3, Machine 4, Machine 24, Machine 27, Machine 32, Machine 33, Machine 50, Machine 51, Machine 52, Machine 55, Machine 57, Machine 58, Machine 62, Machine 64, Machine 65, Machine 68, Machine 72, Machine 73, Machine 77, Machine 83, Machine 85, Machine 86, Machine 89, Machine 90, Machine 93, Machine 94, Machine 98, Machine 99, Machine 100, Machine 105, Machine 107, Machine 114, Machine 118, Machine 122, Machine 125, Machine 130, Machine 131, Machine 146, Machine 151, Machine 152, Machine 155, Machine 157, Machine 165, Machine 168, Machine 178, Machine 181, Machine 182, Machine 185, Machine 187, Machine 191]            
Humans: [Human 0, Human 1, Human 2, Human 5, Human 6, Human 7, Human 8, Human 9, Human 10, Human 11, Human 12, Human 13, Human 14, Human 15, Human 16, Human 17, Human 18, Human 19, Human 20, Human 21, Human 22, Human 23, Human 25, Human 26, Human 28, Human 29, Human 30

> Create a group that contains all the machine (RL) agents.


#### PettingZoo environment wrapper

In [9]:
group = {'agents': [str(machine.id) for machine in env.machine_agents]}

env = PettingZooWrapper(
    env=env,
    use_mask=True,
    categorical_actions=True,
    done_on_any = False,
    group_map=group,
    device=device
)

#### Transforms

Here we instatiate a <code style="color:white">RewardSum</code> transformer that will sum rewards over episode.

In [10]:
env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

The <code style="color:white">check_env_specs()</code> function runs a small rollout and compared it output against the environment specs. It will raise an error if the specs aren't properly defined.

In [ ]:
check_env_specs(env)
env.reset()

self.day is:  100 




2025-02-04 13:08:34,373 [torchrl][INFO] check_env_specs succeeded!


self.day is:  100 


self.day is:  100 




TensorDict(
    fields={
        agents: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                episode_reward: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                mask: Tensor(shape=torch.Size([50]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([50, 4]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([50, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([50]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        terminated: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_

#### Policy network

> Instantiate an `MPL` that can be used in multi-agent contexts.

In [12]:
net = MultiAgentMLP(
        n_agent_inputs=env.observation_spec["agents", "observation"].shape[-1],
        n_agent_outputs=env.action_spec.space.n,
        n_agents=env.n_agents,
        centralised=False,
        share_params=False,
        device=device,
        depth=mlp_depth,
        num_cells=mlp_cells,
        activation_class=nn.ReLU,
    )

In [13]:
module = TensorDictModule(
        net, in_keys=[("agents", "observation")], out_keys=[("agents", "action_value")]
)

In [14]:
value_module = QValueModule(
    action_value_key=("agents", "action_value"),
    out_keys=[
        env.action_key,
        ("agents", "action_value"),
        ("agents", "chosen_action_value"),
    ],
    spec=env.action_spec,
    action_space=None,
)

qnet = SafeSequential(module, value_module)

In [15]:
qnet_explore = TensorDictSequential(
    qnet,
    EGreedyModule(
        eps_init=eps_init,
        eps_end=eps_end,
        annealing_num_steps=int(total_frames * exploration_fraction),
        action_key=env.action_key,
        spec=env.action_spec,
    ),
)

#### Collector

In [16]:
collector = SyncDataCollector(
        env,
        qnet_explore,
        device=device,
        storing_device=device,
        frames_per_batch=frames_per_batch,
        total_frames=total_frames,
    )

#### Replay buffer

In [17]:
replay_buffer = TensorDictReplayBuffer(
        storage=LazyTensorStorage(memory_size, device=device),
        sampler=SamplerWithoutReplacement(),
        batch_size=minibatch_size,
    )

#### DQN loss function

In [18]:
loss_module = DQNLoss(qnet, delay_value=True)

loss_module.set_keys(
        action_value=("agents", "action_value"),
        action=env.action_key,
        value=("agents", "chosen_action_value"),
        reward=env.reward_key,
        done=("agents", "done"),
        terminated=("agents", "terminated"),
)

loss_module.make_value_estimator(ValueEstimators.TD0, gamma=gamma)
target_net_updater = SoftUpdate(loss_module, eps=eps)

optim = torch.optim.Adam(loss_module.parameters(), lr)

#### Training loop

In [ ]:
for i, tensordict_data in enumerate(collector):
    torchrl_logger.info(f"\nIteration {i}")

    current_frames = tensordict_data.numel()
    data_view = tensordict_data.reshape(-1)
    replay_buffer.extend(data_view)
    
    training_tds = []

    ## Update the policies of the learning agents
    for _ in range(num_epochs):
        for _ in range(frames_per_batch // minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)
            training_tds.append(loss_vals.detach())

            loss_value = loss_vals["loss"]

            loss_value.backward()

            total_norm = torch.nn.utils.clip_grad_norm_(loss_module.parameters(), max_grad_norm)
            training_tds[-1].set("grad_norm", total_norm.mean())

            optim.step()
            optim.zero_grad()
        target_net_updater.step()

    qnet_explore[1].step(frames=current_frames)  # Update exploration annealing
    collector.update_policy_weights_()
    
    training_tds = torch.stack(training_tds) 

collector.shutdown()

self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is:  100 


self.day is: 

2025-02-04 13:08:54,813 [torchrl][INFO] 
Iteration 0


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is:  102 


self.day is: 

2025-02-04 13:09:27,408 [torchrl][INFO] 
Iteration 1


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is:  104 


self.day is: 

2025-02-04 13:09:41,817 [torchrl][INFO] 
Iteration 2


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is:  106 


self.day is: 

2025-02-04 13:10:25,264 [torchrl][INFO] 
Iteration 3


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is:  108 


self.day is: 

2025-02-04 13:10:51,079 [torchrl][INFO] 
Iteration 4


In [20]:
env.plot_results()